In [9]:
import pandas as pd
import numpy as np
import math

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 11, Finished, Available, Finished)

In [10]:
#Load datasets

S0030=pd.read_csv('abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/S0030.csv')
S0094=pd.read_csv('abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/S0094.csv')
S0142=pd.read_csv('abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/S0142.csv')


S0030_for=pd.read_csv('abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/FORECAST_S0030.csv')
S0094_for=pd.read_csv('abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/FORECAST_S0094.csv')
S0142_for=pd.read_csv('abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/FORECAST_S0142.csv')

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 12, Finished, Available, Finished)

## S0030

In [11]:
# Inicializar uma lista para armazenar os resultados
last_stock = []
product = S0030['product_id'].unique()

for product_id in product:
    # Filtrar os dados para o produto atual
    product_data = S0030[S0030['product_id'] == product_id]

    # Verificar se há pelo menos 2 registros
    if len(product_data) >= 2:
        # Selecionar os dois últimos valores
        last_row = product_data.iloc[-1]
        last_row_2 = product_data.iloc[-2]

        # Se o último valor for NaN, procurar o último valor não NaN
        if pd.isna(last_row['stock']):
            last_row = product_data[~product_data['stock'].isna()].iloc[-1]

        # Se o penúltimo valor for NaN, procurar o penúltimo valor não NaN
        if pd.isna(last_row_2['stock']):
            last_row_2 = product_data[~product_data['stock'].isna()].iloc[-2]

        # Adicionar o resultado à lista
        last_stock.append({
            'product_id': product_id,
            'last_week': last_row['week'],
            'last_stock': last_row['stock'],
            'last_week_2': last_row_2['week'],
            'last_stock_2': last_row_2['stock']
        })
    else:
        # Se houver menos de 2 registros, adicionar NaN para ambos os valores
        last_stock.append({
            'product_id': product_id,
            'last_week': None,
            'last_stock': None,
            'last_week_2': None,
            'last_stock_2': None
        })

# Criar um DataFrame a partir dos resultados
last_stock_df = pd.DataFrame(last_stock)
last_stock_df

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 13, Finished, Available, Finished)

,product_id,last_week,last_stock,last_week_2,last_stock_2
0,P0015,39.0,10.0,38.0,10.0
1,P0394,6.0,0.0,5.0,1.0
2,P0436,39.0,17.0,38.0,18.0
3,P0438,4.0,1.0,3.0,1.0
4,P0453,39.0,0.0,38.0,15.0
...,...,...,...,...,...
92,P0747,35.0,0.0,34.0,1.0
93,P0559,39.0,1.0,38.0,1.0
94,P0658,39.0,3.0,38.0,3.0
95,P0570,39.0,2.0,38.0,2.0


In [12]:
S0030_2_dt= pd.merge(S0030_for, last_stock_df, left_on='unique_id', right_on='product_id', how='left')

columns_order = [
    'product_id', 'last_week_2', 'last_stock_2', 'last_week', 'last_stock', 
    'ds', 'AutoARIMA', 'AutoARIMA-lo-90', 'AutoARIMA-hi-90']

S0030_2_dt = S0030_2_dt[columns_order]

S0030_2_dt

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 14, Finished, Available, Finished)

,product_id,last_week_2,last_stock_2,last_week,last_stock,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
0,P0015,38.0,10.0,39.0,10.0,2019-10-07,0.366563,-1.479692,2.212818
1,P0015,38.0,10.0,39.0,10.0,2019-10-14,0.430045,-1.486416,2.346506
2,P0026,38.0,8.0,39.0,8.0,2019-10-07,0.000000,-0.879211,0.879211
3,P0026,38.0,8.0,39.0,8.0,2019-10-14,0.000000,-0.879211,0.879211
4,P0035,38.0,13.0,39.0,13.0,2019-10-07,0.565303,-1.518490,2.649097
...,...,...,...,...,...,...,...,...,...
123,P0711,38.0,26.0,39.0,25.0,2019-10-14,0.000000,-3.162672,3.162672
124,P0718,38.0,1.0,39.0,5.0,2019-10-07,0.341611,-0.762704,1.445927
125,P0718,38.0,1.0,39.0,5.0,2019-10-14,0.341486,-0.762957,1.445930
126,P0729,38.0,2.0,39.0,2.0,2019-10-07,0.000000,0.000000,0.000000


## S0094

In [13]:
# Inicializar uma lista para armazenar os resultados
last_stock = []
product = S0094['product_id'].unique()

for product_id in product:
    # Filtrar os dados para o produto atual
    product_data = S0094[S0094['product_id'] == product_id]

    # Verificar se há pelo menos 2 registros
    if len(product_data) >= 2:
        # Selecionar os dois últimos valores
        last_row = product_data.iloc[-1]
        last_row_2 = product_data.iloc[-2]

        # Se o último valor for NaN, procurar o último valor não NaN
        if pd.isna(last_row['stock']):
            last_row = product_data[~product_data['stock'].isna()].iloc[-1]

        # Se o penúltimo valor for NaN, procurar o penúltimo valor não NaN
        if pd.isna(last_row_2['stock']):
            last_row_2 = product_data[~product_data['stock'].isna()].iloc[-2]

        # Adicionar o resultado à lista
        last_stock.append({
            'product_id': product_id,
            'last_week': last_row['week'],
            'last_stock': last_row['stock'],
            'last_week_2': last_row_2['week'],
            'last_stock_2': last_row_2['stock']
        })
    else:
        # Se houver menos de 2 registros, adicionar NaN para ambos os valores
        last_stock.append({
            'product_id': product_id,
            'last_week': None,
            'last_stock': None,
            'last_week_2': None,
            'last_stock_2': None
        })

# Criar um DataFrame a partir dos resultados
last_stock_df = pd.DataFrame(last_stock)
last_stock_df

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 15, Finished, Available, Finished)

,product_id,last_week,last_stock,last_week_2,last_stock_2
0,P0151,39.0,1.0,38.0,1.0
1,P0333,39.0,6.0,38.0,12.0
2,P0454,39.0,9.0,38.0,10.0
3,P0071,3.0,0.0,2.0,3.0
4,P0226,14.0,9.0,13.0,9.0
...,...,...,...,...,...
406,P0494,39.0,16.0,38.0,17.0
407,P0676,39.0,1.0,39.0,11.0
408,P0634,39.0,8.0,39.0,4.0
409,P0270,NaN,NaN,NaN,NaN


In [14]:
S0094_2_dt= pd.merge(S0094_for, last_stock_df, left_on='unique_id', right_on='product_id', how='left')

columns_order = [
    'product_id', 'last_week_2', 'last_stock_2', 'last_week', 'last_stock', 
    'ds', 'AutoARIMA', 'AutoARIMA-lo-90', 'AutoARIMA-hi-90']

S0094_2_dt = S0094_2_dt[columns_order]

S0094_2_dt

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 16, Finished, Available, Finished)

,product_id,last_week_2,last_stock_2,last_week,last_stock,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
0,P0005,38.0,6.0,39.0,6.0,2019-10-07,0.097343,-0.438197,0.632883
1,P0005,38.0,6.0,39.0,6.0,2019-10-14,0.044856,-0.491544,0.581255
2,P0007,37.0,1.0,38.0,0.0,2019-10-07,0.228821,-0.490328,0.947970
3,P0008,38.0,12.0,39.0,12.0,2019-10-07,0.882989,-1.248524,3.014502
4,P0008,38.0,12.0,39.0,12.0,2019-10-14,0.882638,-1.249248,3.014524
...,...,...,...,...,...,...,...,...,...
604,P0741,38.0,14.0,39.0,14.0,2019-10-14,0.000000,-0.764841,0.764841
605,P0742,38.0,4.0,39.0,4.0,2019-10-07,-0.000002,-0.994910,0.994907
606,P0742,38.0,4.0,39.0,4.0,2019-10-14,0.000000,-1.106395,1.106395
607,P0748,38.0,16.0,39.0,16.0,2019-10-07,0.000000,-1.965976,1.965976


## S0142

In [15]:
# Inicializar uma lista para armazenar os resultados
last_stock = []
product = S0142['product_id'].unique()

for product_id in product:
    # Filtrar os dados para o produto atual
    product_data = S0142[S0142['product_id'] == product_id]

    # Verificar se há pelo menos 2 registros
    if len(product_data) >= 2:
        # Selecionar os dois últimos valores
        last_row = product_data.iloc[-1]
        last_row_2 = product_data.iloc[-2]

        # Se o último valor for NaN, procurar o último valor não NaN
        if pd.isna(last_row['stock']):
            last_row = product_data[~product_data['stock'].isna()].iloc[-1]

        # Se o penúltimo valor for NaN, procurar o penúltimo valor não NaN
        if pd.isna(last_row_2['stock']):
            last_row_2 = product_data[~product_data['stock'].isna()].iloc[-2]

        # Adicionar o resultado à lista
        last_stock.append({
            'product_id': product_id,
            'last_week': last_row['week'],
            'last_stock': last_row['stock'],
            'last_week_2': last_row_2['week'],
            'last_stock_2': last_row_2['stock']
        })
    else:
        # Se houver menos de 2 registros, adicionar NaN para ambos os valores
        last_stock.append({
            'product_id': product_id,
            'last_week': None,
            'last_stock': None,
            'last_week_2': None,
            'last_stock_2': None
        })

# Criar um DataFrame a partir dos resultados
last_stock_df = pd.DataFrame(last_stock)
last_stock_df

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 17, Finished, Available, Finished)

,product_id,last_week,last_stock,last_week_2,last_stock_2
0,P0652,39.0,14.0,38.0,14.0
1,P0490,39.0,4.0,38.0,4.0
2,P0491,8.0,0.0,7.0,1.0
3,P0103,39.0,72.0,38.0,87.0
4,P0499,39.0,33.0,38.0,45.0
...,...,...,...,...,...
287,P0570,39.0,1.0,38.0,1.0
288,P0117,39.0,6.0,38.0,6.0
289,P0676,39.0,12.0,39.0,12.0
290,P0059,39.0,5.0,38.0,6.0


In [16]:
S0142_2_dt= pd.merge(S0142_for, last_stock_df, left_on='unique_id', right_on='product_id', how='left')

columns_order = [
    'product_id', 'last_week_2', 'last_stock_2', 'last_week', 'last_stock', 
    'ds', 'AutoARIMA', 'AutoARIMA-lo-90', 'AutoARIMA-hi-90']

S0142_2_dt = S0142_2_dt[columns_order]

S0142_2_dt = S0142_2_dt.drop_duplicates()

S0142_2_dt

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 18, Finished, Available, Finished)

,product_id,last_week_2,last_stock_2,last_week,last_stock,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
0,P0005,38.0,2.0,39.0,2.0,2019-10-07,0.000000e+00,0.000000,0.000000
1,P0005,38.0,2.0,39.0,2.0,2019-10-14,0.000000e+00,0.000000,0.000000
2,P0007,38.0,3.0,39.0,3.0,2019-10-07,-5.302479e-10,-0.490007,0.490007
3,P0007,38.0,3.0,39.0,3.0,2019-10-14,0.000000e+00,-0.523981,0.523981
4,P0009,38.0,12.0,39.0,12.0,2019-10-07,3.365103e-01,-1.398520,2.071540
...,...,...,...,...,...,...,...,...,...
442,P0742,38.0,5.0,39.0,5.0,2019-10-14,0.000000e+00,-0.660589,0.660589
443,P0747,38.0,16.0,39.0,16.0,2019-10-07,2.971069e-01,-5.019232,5.613446
444,P0747,38.0,16.0,39.0,16.0,2019-10-14,3.162127e-02,-5.314743,5.377986
445,P0748,38.0,19.0,39.0,18.0,2019-10-07,3.014096e-01,-0.668608,1.271427


## Final Table

## S0030

In [17]:
S0030_2_dt['ds'] = pd.to_datetime(S0030_2_dt['ds'])


# Encontrar o índice da linha com a data máxima para cada combinação de 'product_id'
idx = S0030_2_dt.groupby('product_id')['ds'].idxmax()

# Filtrar o DataFrame para manter apenas essas linhas
S0030_fin = S0030_2_dt.loc[idx]

# Resetar o índice se desejar
S0030_fin = S0030_fin.reset_index(drop=True)

S0030_fin

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 19, Finished, Available, Finished)

,product_id,last_week_2,last_stock_2,last_week,last_stock,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
0,P0015,38.0,10.0,39.0,10.0,2019-10-14,0.430045,-1.486416,2.346506
1,P0026,38.0,8.0,39.0,8.0,2019-10-14,0.000000,-0.879211,0.879211
2,P0035,38.0,13.0,39.0,13.0,2019-10-14,0.717025,-1.374442,2.808492
3,P0051,38.0,2.0,39.0,2.0,2019-10-14,0.423664,-5.463986,6.311315
4,P0055,38.0,10.0,39.0,10.0,2019-10-14,0.000000,-1.203471,1.203471
...,...,...,...,...,...,...,...,...,...
61,P0694,38.0,3.0,39.0,2.0,2019-10-14,1.360877,-0.203475,2.925228
62,P0709,38.0,4.0,39.0,9.0,2019-10-14,5.156817,-3.570445,13.884078
63,P0711,38.0,26.0,39.0,25.0,2019-10-14,0.000000,-3.162672,3.162672
64,P0718,38.0,1.0,39.0,5.0,2019-10-14,0.341486,-0.762957,1.445930


In [18]:
# Calcular Quant_needed

S0030_frcst=S0030_fin.copy()

S0030_frcst['Quant_needed'] = S0030_frcst['last_stock'] - (
    (0.1 * S0030_2_dt['AutoARIMA']) + 
    S0030_2_dt['AutoARIMA'] + 
    (0.1 * S0030_fin['AutoARIMA']) + 
    S0030_fin['AutoARIMA']
)

# Selecionar e reordenar as colunas
result = S0030_frcst[['product_id', 'last_week_2', 'last_stock_2', 'last_week', 'last_stock', 'ds', 'AutoARIMA', 'AutoARIMA-lo-90', 'AutoARIMA-hi-90', 'Quant_needed']]

result

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 20, Finished, Available, Finished)

,product_id,last_week_2,last_stock_2,last_week,last_stock,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90,Quant_needed
0,P0015,38.0,10.0,39.0,10.0,2019-10-14,0.430045,-1.486416,2.346506,9.123731
1,P0026,38.0,8.0,39.0,8.0,2019-10-14,0.000000,-0.879211,0.879211,7.526950
2,P0035,38.0,13.0,39.0,13.0,2019-10-14,0.717025,-1.374442,2.808492,12.211272
3,P0051,38.0,2.0,39.0,2.0,2019-10-14,0.423664,-5.463986,6.311315,1.533969
4,P0055,38.0,10.0,39.0,10.0,2019-10-14,0.000000,-1.203471,1.203471,9.378166
...,...,...,...,...,...,...,...,...,...,...
61,P0694,38.0,3.0,39.0,2.0,2019-10-14,1.360877,-0.203475,2.925228,-0.330795
62,P0709,38.0,4.0,39.0,9.0,2019-10-14,5.156817,-3.570445,13.884078,3.324918
63,P0711,38.0,26.0,39.0,25.0,2019-10-14,0.000000,-3.162672,3.162672,24.997417
64,P0718,38.0,1.0,39.0,5.0,2019-10-14,0.341486,-0.762957,1.445930,-0.964220


In [19]:
predictions_S0030= result.copy()
predictions_S0030['ds'] = pd.to_datetime(predictions_S0030['ds'])

predictions_S0030['current_date'] = predictions_S0030['ds'] - pd.DateOffset(weeks=2)
predictions_S0030['current_stock']=predictions_S0030['last_stock']
predictions_S0030['predicted_sales']=predictions_S0030['AutoARIMA']
predictions_S0030['future_week']=predictions_S0030['ds']
predictions_S0030['need_buy'] = predictions_S0030['Quant_needed'].apply(lambda x: 'YES' if x < 0 else 'NO')

predictions_S0030['quant_needed'] = np.where(predictions_S0030['Quant_needed'] > 0, 0, predictions_S0030['Quant_needed'])
predictions_S0030['quant_needed'] = predictions_S0030['quant_needed'].apply(lambda x: math.ceil(abs(x)))

columns=['last_week_2','last_stock_2','ds','last_stock','ds','AutoARIMA','last_week','AutoARIMA-lo-90','AutoARIMA-hi-90','Quant_needed']
predictions_S0030.drop(columns, axis=1,inplace=True)
predictions_S0030



StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 21, Finished, Available, Finished)

,product_id,current_date,current_stock,predicted_sales,future_week,need_buy,quant_needed
0,P0015,2019-09-30,10.0,0.430045,2019-10-14,NO,0
1,P0026,2019-09-30,8.0,0.000000,2019-10-14,NO,0
2,P0035,2019-09-30,13.0,0.717025,2019-10-14,NO,0
3,P0051,2019-09-30,2.0,0.423664,2019-10-14,NO,0
4,P0055,2019-09-30,10.0,0.000000,2019-10-14,NO,0
...,...,...,...,...,...,...,...
61,P0694,2019-09-30,2.0,1.360877,2019-10-14,YES,1
62,P0709,2019-09-30,9.0,5.156817,2019-10-14,NO,0
63,P0711,2019-09-30,25.0,0.000000,2019-10-14,NO,0
64,P0718,2019-09-30,5.0,0.341486,2019-10-14,YES,1


In [12]:
predictions_S0030.to_csv('abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/FT_S0030.csv')

StatementMeta(, 978ad3f3-7938-4715-9183-c10f10b18aa8, 14, Finished, Available, Finished)

## S0094

In [25]:
S0094_2_dt['ds'] = pd.to_datetime(S0094_2_dt['ds'])


# Encontrar o índice da linha com a data máxima para cada combinação de 'product_id'
idx = S0094_2_dt.groupby('product_id')['ds'].idxmax()

# Filtrar o DataFrame para manter apenas essas linhas
S0094_fin = S0094_2_dt.loc[idx]

# Resetar o índice se desejar
S0094_fin = S0094_fin.reset_index(drop=True)

S0094_fin

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 27, Finished, Available, Finished)

,product_id,last_week_2,last_stock_2,last_week,last_stock,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
1,P0005,38.0,6.0,39.0,6.0,2019-10-14,0.044856,-0.491544,0.581255
2,P0007,37.0,1.0,38.0,0.0,2019-10-07,0.228821,-0.490328,0.947970
4,P0008,38.0,12.0,39.0,12.0,2019-10-14,0.882638,-1.249248,3.014524
6,P0009,38.0,5.0,39.0,3.0,2019-10-14,0.801551,-0.767815,2.370917
8,P0012,38.0,8.0,39.0,7.0,2019-10-14,0.000000,-0.822427,0.822427
...,...,...,...,...,...,...,...,...,...
600,P0738,38.0,9.0,39.0,9.0,2019-10-14,0.000000,-0.854691,0.854691
602,P0739,38.0,18.0,39.0,14.0,2019-10-14,2.730788,-16.065290,21.526865
604,P0741,38.0,14.0,39.0,14.0,2019-10-14,0.000000,-0.764841,0.764841
606,P0742,38.0,4.0,39.0,4.0,2019-10-14,0.000000,-1.106395,1.106395


In [26]:
# Calcular Quant_needed

S0094_frcst=S0094_fin.copy()

S0094_frcst['Quant_needed'] = S0094_frcst['last_stock'] - (
    (0.1 * S0094_frcst['AutoARIMA']) + 
    S0094_frcst['AutoARIMA'] + 
    (0.1 * S0094_frcst['AutoARIMA']) + 
    S0094_frcst['AutoARIMA']
)

# Selecionar e reordenar as colunas
result1 = S0094_frcst[['product_id', 'last_week_2', 'last_stock_2', 'last_week', 'last_stock', 'ds', 'AutoARIMA', 'AutoARIMA-lo-90', 'AutoARIMA-hi-90', 'Quant_needed']]

result1

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 28, Finished, Available, Finished)

,product_id,last_week_2,last_stock_2,last_week,last_stock,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90,Quant_needed
1,P0005,38.0,6.0,39.0,6.0,2019-10-14,0.044856,-0.491544,0.581255,5.901318
2,P0007,37.0,1.0,38.0,0.0,2019-10-07,0.228821,-0.490328,0.947970,-0.503407
4,P0008,38.0,12.0,39.0,12.0,2019-10-14,0.882638,-1.249248,3.014524,10.058197
6,P0009,38.0,5.0,39.0,3.0,2019-10-14,0.801551,-0.767815,2.370917,1.236588
8,P0012,38.0,8.0,39.0,7.0,2019-10-14,0.000000,-0.822427,0.822427,7.000000
...,...,...,...,...,...,...,...,...,...,...
600,P0738,38.0,9.0,39.0,9.0,2019-10-14,0.000000,-0.854691,0.854691,9.000000
602,P0739,38.0,18.0,39.0,14.0,2019-10-14,2.730788,-16.065290,21.526865,7.992266
604,P0741,38.0,14.0,39.0,14.0,2019-10-14,0.000000,-0.764841,0.764841,14.000000
606,P0742,38.0,4.0,39.0,4.0,2019-10-14,0.000000,-1.106395,1.106395,4.000000


In [15]:
predictions_S0094= result1.copy()
predictions_S0094['ds'] = pd.to_datetime(predictions_S0094['ds'])

predictions_S0094['current_date'] = predictions_S0094['ds'] - pd.DateOffset(weeks=2)
predictions_S0094['current_stock']=predictions_S0094['last_stock']
predictions_S0094['predicted_sales']=predictions_S0094['AutoARIMA']
predictions_S0094['future_week']=predictions_S0094['ds']
predictions_S0094['need_buy'] = predictions_S0094['Quant_needed'].apply(lambda x: 'YES' if x < 0 else 'NO')

predictions_S0094['quant_needed'] = np.where(predictions_S0094['Quant_needed'] > 0, 0, predictions_S0094['Quant_needed'])
predictions_S0094['quant_needed'] = predictions_S0094['quant_needed'].apply(
    lambda x: math.ceil(abs(x)) if pd.notna(x) else np.nan
)
columns=['last_week_2','last_stock_2','ds','last_stock','ds','AutoARIMA','last_week','AutoARIMA-lo-90','AutoARIMA-hi-90','Quant_needed']
predictions_S0094.drop(columns, axis=1, inplace= True)
predictions_S0094


StatementMeta(, 978ad3f3-7938-4715-9183-c10f10b18aa8, 17, Finished, Available, Finished)

,product_id,current_date,current_stock,predicted_sales,future_week,need_buy,quant_needed
0,P0005,2019-09-30,6.0,0.044856,2019-10-14,NO,0.0
1,P0007,2019-09-23,0.0,0.228821,2019-10-07,YES,1.0
2,P0008,2019-09-30,12.0,0.882638,2019-10-14,NO,0.0
3,P0009,2019-09-30,3.0,0.801551,2019-10-14,NO,0.0
4,P0012,2019-09-30,7.0,0.000000,2019-10-14,NO,0.0
...,...,...,...,...,...,...,...
302,P0738,2019-09-30,9.0,0.000000,2019-10-14,NO,0.0
303,P0739,2019-09-30,14.0,2.730788,2019-10-14,NO,0.0
304,P0741,2019-09-30,14.0,0.000000,2019-10-14,NO,0.0
305,P0742,2019-09-30,4.0,0.000000,2019-10-14,NO,0.0


In [16]:
predictions_S0094.to_csv('abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/FT_S0094.csv')

StatementMeta(, 978ad3f3-7938-4715-9183-c10f10b18aa8, 18, Finished, Available, Finished)

## S0142

In [34]:
S0142_2_dt['ds'] = pd.to_datetime(S0142_2_dt['ds'])


# Encontrar o índice da linha com a data máxima para cada combinação de 'product_id'
idx = S0142_2_dt.groupby('product_id')['ds'].idxmax()

# Filtrar o DataFrame para manter apenas essas linhas
S0142_fin = S0142_2_dt.loc[idx]

# Resetar o índice se desejar
S0142_fin = S0142_fin.reset_index(drop=True)

S0142_fin

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 36, Finished, Available, Finished)

,product_id,last_week_2,last_stock_2,last_week,last_stock,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
0,P0005,38.0,2.0,39.0,2.0,2019-10-14,0.000000,0.000000,0.000000
1,P0007,38.0,3.0,39.0,3.0,2019-10-14,0.000000,-0.523981,0.523981
2,P0009,38.0,12.0,39.0,12.0,2019-10-14,0.333620,-1.404085,2.071324
3,P0012,NaN,NaN,NaN,NaN,2019-10-14,0.000000,0.000000,0.000000
4,P0015,38.0,16.0,39.0,16.0,2019-10-14,0.278899,-1.481654,2.039451
...,...,...,...,...,...,...,...,...,...
219,P0733,38.0,13.0,39.0,12.0,2019-10-14,0.716518,-4.002003,5.435040
220,P0741,38.0,3.0,39.0,3.0,2019-10-14,0.000000,-0.286332,0.286332
221,P0742,38.0,5.0,39.0,5.0,2019-10-14,0.000000,-0.660589,0.660589
222,P0747,38.0,16.0,39.0,16.0,2019-10-14,0.031621,-5.314743,5.377986


In [35]:
# Calcular Quant_needed

S0142_frcst=S0142_fin.copy()

S0142_frcst['Quant_needed'] = S0142_frcst['last_stock'] - (
    (0.1 * S0142_frcst['AutoARIMA']) + 
    S0142_frcst['AutoARIMA'] + 
    (0.1 * S0142_frcst['AutoARIMA']) + 
    S0142_frcst['AutoARIMA']
)

# Selecionar e reordenar as colunas
result2 = S0142_frcst[['product_id', 'last_week_2', 'last_stock_2', 'last_week', 'last_stock', 'ds', 'AutoARIMA', 'AutoARIMA-lo-90', 'AutoARIMA-hi-90', 'Quant_needed']]

result2

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 37, Finished, Available, Finished)

,product_id,last_week_2,last_stock_2,last_week,last_stock,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90,Quant_needed
0,P0005,38.0,2.0,39.0,2.0,2019-10-14,0.000000,0.000000,0.000000,2.000000
1,P0007,38.0,3.0,39.0,3.0,2019-10-14,0.000000,-0.523981,0.523981,3.000000
2,P0009,38.0,12.0,39.0,12.0,2019-10-14,0.333620,-1.404085,2.071324,11.266037
3,P0012,NaN,NaN,NaN,NaN,2019-10-14,0.000000,0.000000,0.000000,NaN
4,P0015,38.0,16.0,39.0,16.0,2019-10-14,0.278899,-1.481654,2.039451,15.386423
...,...,...,...,...,...,...,...,...,...,...
219,P0733,38.0,13.0,39.0,12.0,2019-10-14,0.716518,-4.002003,5.435040,10.423660
220,P0741,38.0,3.0,39.0,3.0,2019-10-14,0.000000,-0.286332,0.286332,3.000000
221,P0742,38.0,5.0,39.0,5.0,2019-10-14,0.000000,-0.660589,0.660589,5.000000
222,P0747,38.0,16.0,39.0,16.0,2019-10-14,0.031621,-5.314743,5.377986,15.930433


In [36]:
predictions_S0142= result2.copy()
predictions_S0142['ds'] = pd.to_datetime(predictions_S0142['ds'])

predictions_S0142['current_date'] = predictions_S0142['ds'] - pd.DateOffset(weeks=2)
predictions_S0142['current_stock']=predictions_S0142['last_stock']
predictions_S0142['predicted_sales']=predictions_S0142['AutoARIMA'].apply(lambda x: f'{x:.8f}')
predictions_S0142['future_week']=predictions_S0142['ds']
predictions_S0142['need_buy'] = predictions_S0142['Quant_needed'].apply(lambda x: 'YES' if x < 0 else 'NO')

predictions_S0142['quant_needed'] = np.where(predictions_S0142['Quant_needed'] > 0, 0, predictions_S0142['Quant_needed'])
predictions_S0142['quant_needed'] = predictions_S0142['quant_needed'].apply(
    lambda x: math.ceil(abs(x)) if pd.notna(x) else np.nan
)
columns=['last_week_2','last_stock_2','ds','last_stock','ds','AutoARIMA','last_week','AutoARIMA-lo-90','AutoARIMA-hi-90','Quant_needed']
predictions_S0142.drop(columns, axis=1, inplace=True)

predictions_S0142


StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 38, Finished, Available, Finished)

,product_id,current_date,current_stock,predicted_sales,future_week,need_buy,quant_needed
0,P0005,2019-09-30,2.0,0.00000000,2019-10-14,NO,0.0
1,P0007,2019-09-30,3.0,0.00000000,2019-10-14,NO,0.0
2,P0009,2019-09-30,12.0,0.33361956,2019-10-14,NO,0.0
3,P0012,2019-09-30,NaN,0.00000000,2019-10-14,NO,NaN
4,P0015,2019-09-30,16.0,0.27889884,2019-10-14,NO,0.0
...,...,...,...,...,...,...,...
219,P0733,2019-09-30,12.0,0.71651816,2019-10-14,NO,0.0
220,P0741,2019-09-30,3.0,0.00000000,2019-10-14,NO,0.0
221,P0742,2019-09-30,5.0,0.00000000,2019-10-14,NO,0.0
222,P0747,2019-09-30,16.0,0.03162127,2019-10-14,NO,0.0


In [37]:
predictions_S0142.to_csv('abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/FT_S0142.csv')

StatementMeta(, 8fab9407-6417-48dd-bb78-dcc50675539a, 39, Finished, Available, Finished)